In [1]:
!pwd

/home/qm


In [2]:
import torch
import transformers
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

In [3]:
model_id = "/data/qm/huggingface/Meta-Llama-3-8B-Instruct"

device_map = "auto"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# load model from huggingface
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    use_cache=True,
    device_map=device_map
)

# load tokenizer from huggingface
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot, always able to provide friendly and helpful answers."
    },
    {
        "role": "user",
        "content": "What athlete from the 1992 U.S. Olympic 'Dream Team' had a role in the movie 'Space Jam'?"
    },
    {
        "role": "assistant",
        "content": "The athlete from the 1992 U.S. Olympic 'Dream Team' who had a role in the movie 'Space Jam' is Michael Jordan. He played the role of the basketball player \"Lucas North\" in the film, which was released in 1996. The movie was based on a fictional basketball team called the \"Space Jam,\" which was created by Warner Bros. And NBC Universal for the release of the basketball movie \"Space Jam\" in 1996."
    },
    {
        "role": "user",
        "content": "Can you give me more information about that?"
    }
]
model_inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
generated_ids = model.generate(model_inputs, max_new_tokens=512, temperature=0.7)
result = tokenizer.batch_decode(generated_ids)
print(result)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
2024-06-26 09:40:54.985804: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-26 09:40:56.038531: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a friendly chatbot, always able to provide friendly and helpful answers.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhat athlete from the 1992 U.S. Olympic \'Dream Team\' had a role in the movie \'Space Jam\'?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThe athlete from the 1992 U.S. Olympic \'Dream Team\' who had a role in the movie \'Space Jam\' is Michael Jordan. He played the role of the basketball player "Lucas North" in the film, which was released in 1996. The movie was based on a fictional basketball team called the "Space Jam," which was created by Warner Bros. And NBC Universal for the release of the basketball movie "Space Jam" in 1996.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nCan you give me more information about that?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nMichael Jordan\'s role in Space Jam was a significant one. In the movie, he plays the chara

In [9]:
mark1 = '<|start_header_id|>assistant<|end_header_id|>'
mark2 = '<|eot_id|>'
_result = result[0]
print(_result[_result.find(mark1) + len(mark1):_result.rfind(mark2)].strip())

The athlete from the 1992 U.S. Olympic 'Dream Team' who had a role in the movie 'Space Jam' is Michael Jordan. He played the role of the basketball player "Lucas North" in the film, which was released in 1996. The movie was based on a fictional basketball team called the "Space Jam," which was created by Warner Bros. And NBC Universal for the release of the basketball movie "Space Jam" in 1996.<|eot_id|><|start_header_id|>user<|end_header_id|>

Can you give me more information about that?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Michael Jordan's role in Space Jam was a significant one. In the movie, he plays the character of "Lucky" Lofton, not Lucas North. Lucky is a fictionalized version of himself, a retired basketball player who is lured back to the court by Looney Tunes characters Bugs Bunny and Daffy Duck.

The movie follows the story of a group of Looney Tunes characters who are threatened by a group of aliens who are stealing their talents. The aliens, led by Mr.